# <center> **SIACON**

In [69]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [70]:
import sqlite3
import pandas as pd
import numpy as np
import openpyxl

Ya instalado el SIACON, buscamos la ruta de descarga, en mi caso se descargo en 
programa de archivos carpeta **SIACON**, y con el nombre de **siacon-ng.db**

In [3]:
con = sqlite3.connect('C:/Program Files (x86)/SIACON/siacon-ng.db')

In [33]:
# Imprime lista de tablas
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
# Creating cursor object using connection object
cursor = con.cursor()
# lista de tablas 
tables = []    
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in cursor.fetchall():
    tables.append(table[0])
# lista de tablas        
tables

['cat_ciclo',
 'cat_cultivo',
 'cat_especie',
 'cat_especiepesquero',
 'cat_gruponatural',
 'cat_litoral',
 'cat_modalidad',
 'cat_municipio',
 'cat_producto',
 'cat_clasificacion',
 'cat_tipoagricultura',
 'cat_unidadmedida',
 'cat_variedad',
 'pes_catalogo',
 'peco_cierre_estatal',
 'peco_cierre_municipal',
 'peso_cierre_estatal',
 'pes_elemento',
 'cat_tipo_produccion',
 'cat_generico',
 'cultivo_generico',
 'cat_region',
 'cat_estado',
 'agro_cierre_estatal',
 'cat_mercado',
 'agro_cierre_municipal']

In [85]:
# base municipal
base = pd.read_sql_query("SELECT * from cat_variedad", con)
#base.to_excel("cultivo_generico.xlsx")
base.head()

,idcultivo,idvariedad,idgrupo,idunidadmedida,nomvariedad,variedad
0,5370000,5370120,10001040,200201,Apio invernadero,Apio invernadero
1,5370000,5370130,10001040,200201,Apio malla sombra,Apio malla sombra
2,5400000,5400100,10001030,200201,Arrayán s/clasificar,Arrayan s/clasificar
3,5410000,5410400,10001000,200201,Arroz palay aztecas,Arroz palay aztecas
4,5410000,5410500,10001000,200201,Arroz palay grueso milagro filipino orgánico,Arroz palay grueso milagro filipino organico


In [26]:
base.columns

Index(['anio', 'idestado', 'idddr', 'idcader', 'idmunicipio', 'idciclo',
       'idmodalidad', 'idcultivo', 'idvariedad', 'idunidadmedida',
       'idtipoagricultura', 'idtipoproduccion', 'idmercado', 'sembrada',
       'cosechada', 'siniestrada', 'volumenproduccion', 'valorproduccion',
       'rendimiento', 'preciomediorural'],
      dtype='object')

In [87]:
# label estado
id_estado = pd.read_sql_query("SELECT * from cat_estado", con)
id_estado = id_estado.loc[:, ["idestado", "nomestado"]]
id_estado.columns
id_estado.to_excel('cat_estado.xlsx', sheet_name='estados',index=False)
# label municipios
id_municipio = pd.read_sql_query("SELECT * from cat_municipio", con)
id_municipio.columns
id_municipio.to_excel('cat_municipio.xlsx', sheet_name='municipios',index=False)
# label ciclo
id_ciclo = pd.read_sql_query("SELECT * from cat_ciclo", con)
id_ciclo.columns
id_ciclo.to_excel('cat_ciclo.xlsx', sheet_name='ciclo',index=False)
# label modalidad
id_modalidad = pd.read_sql_query("SELECT * from cat_modalidad", con)
id_modalidad.columns
id_modalidad.to_excel('cat_modalidad.xlsx', sheet_name='modalidad',index=False)

# label cultivo
id_cultivo = pd.read_sql_query("SELECT * from cat_cultivo", con)
id_cultivo.columns
id_cultivo.to_excel('cat_cultivo.xlsx', sheet_name='cultivo',index=False)

# label cultivo
id_variedad = pd.read_sql_query("SELECT * from cat_variedad", con)
id_variedad = id_variedad.drop('idgrupo', axis=1)
id_variedad.to_excel('cat_variedad.xlsx', sheet_name='variedad',index=False)

# label unidad de medida
id_unidadmedida = pd.read_sql_query("SELECT * from cat_unidadmedida", con)
id_unidadmedida.to_excel('cat_unidadmedida.xlsx', sheet_name='unidadmedida',index=False)
id_unidadmedida.columns
# label unidad de medida
id_tipoproduccion = pd.read_sql_query("SELECT * from cat_tipo_produccion", con)
id_tipoproduccion.to_excel('cat_tipoproduccion.xlsx', sheet_name='tipoproduccion',index=False)
id_tipoproduccion.columns
# label tipo agricultura
id_tipoagricultura = pd.read_sql_query("SELECT * from cat_tipoagricultura", con)
id_tipoagricultura.to_excel('cat_tipoagricultura.xlsx', sheet_name='tipoagricultura',index=False)
id_tipoagricultura.columns
# label mercado
id_mercado = pd.read_sql_query("SELECT * from cat_mercado", con)
id_mercado.to_excel('cat_mercado.xlsx', sheet_name='mercado',index=False)
id_mercado.columns

Index(['idmercado', 'nommercado'], dtype='object')

## Unión de bases

In [73]:
base_completa = base.merge(id_estado, on=['idestado'], how='left').\
    merge(id_municipio, on=['idestado', 'idmunicipio'], how='left').\
    merge(id_ciclo, on=['idciclo'], how='left').\
    merge(id_modalidad, on=['idmodalidad'], how='left').\
    merge(id_cultivo, on=['idcultivo', 'idunidadmedida'], how='left').\
    merge(id_variedad, on=['idcultivo', 'idvariedad','idunidadmedida'], how='left').\
    merge(id_unidadmedida, on=['idunidadmedida'], how='left').\
    merge(id_tipoagricultura, on=['idtipoagricultura'], how='left').\
    merge(id_tipoproduccion, on=['idtipoproduccion'], how='left').\
    merge(id_mercado, on=['idmercado'], how='left')

In [75]:
ordercols = ['anio','idddr', 'idcader', 
        'idestado','nomestado', 
        'idmunicipio','nommunicipio',
        'idciclo','nomciclo',
       'idmodalidad','nommodalidad',
       'idcultivo','nomcultivo', 'cultivo',
       'idvariedad','variedad', 'nomvariedad',
       'idunidadmedida','nomunidad','nomcorto',
       'idtipoagricultura', 'nomtipoagricultura',
       'idtipoproduccion', 'nomtipoproduccion',
       'idmercado', 'nommercado',
       'sembrada','cosechada', 'siniestrada', 'volumenproduccion', 
       'valorproduccion','rendimiento', 'preciomediorural']
base_completa = base_completa[ordercols]
base_completa.head()

,anio,idddr,idcader,idestado,nomestado,idmunicipio,nommunicipio,idciclo,nomciclo,idmodalidad,...,nomtipoproduccion,idmercado,nommercado,sembrada,cosechada,siniestrada,volumenproduccion,valorproduccion,rendimiento,preciomediorural
0,2016,1,1,1,Aguascalientes,1,Aguascalientes,1,Otoño-Invierno,1,...,Convencional,10010301,Nacional,765.0,765.0,0.0,20865.75,12424510.84,27.275,595.45
1,2016,1,1,1,Aguascalientes,1,Aguascalientes,1,Otoño-Invierno,1,...,Convencional,10010301,Nacional,35.0,35.0,0.0,945.00,538650.00,27.000,570.00
2,2016,1,1,1,Aguascalientes,1,Aguascalientes,1,Otoño-Invierno,1,...,Convencional,10010301,Nacional,611.0,611.0,0.0,20446.00,12127953.82,33.463,593.17
3,2016,1,1,1,Aguascalientes,1,Aguascalientes,1,Otoño-Invierno,1,...,Convencional,10010301,Nacional,692.0,692.0,0.0,34912.70,20987769.60,50.452,601.15
4,2016,1,1,1,Aguascalientes,1,Aguascalientes,2,Primavera-Verano,1,...,Convencional,10010301,Nacional,28.0,28.0,0.0,746.40,407467.22,26.657,545.91


In [76]:
base_completa.to_excel('agro_cierre_municipal.xlsx', 
                       sheet_name='SIAP',
                       index=False)
